# Оптимизация изображений: MSE и сравнение подходов
В этом ноутбуке мы вычисляем среднеквадратичную ошибку (MSE) между изображениями и сравниваем два метода оптимизации: Монте-Карло и Дифференциальная эволюция.

In [ ]:
import numpy as np
from PIL import Image
import os
import random
from scipy.optimize import differential_evolution
from typing import List, Tuple, Callable, Any


## Функция для расчёта MSE
Ниже определена функция, которая принимает два массива изображений и возвращает их среднеквадратичную ошибку.

In [ ]:
def calculate_mse(img1: np.ndarray, img2: np.ndarray) -> float:
    """Возвращает среднеквадратичную ошибку между двумя изображениями."""
    return np.mean((img1 - img2) ** 2)


## Загрузка изображений
Функция, которая проходит по заданному каталогу и загружает все файлы изображений в виде numpy-массивов.

In [ ]:
def load_images_from_folder(path: str) -> List[np.ndarray]:
    """Сканирует директорию и возвращает список массивов изображений."""
    imgs = []
    for fname in os.listdir(path):
        full_path = os.path.join(path, fname)
        if os.path.isfile(full_path):
            try:
                image = Image.open(full_path)
                imgs.append(np.array(image))
            except Exception as e:
                print(f"Не удалось загрузить {fname}: {e}")
    return imgs


## Классы оптимизаторов
Определяем базовый класс `Optimizer` и два наследника: `MonteCarloOptimizer` и `DifferentialEvolutionOptimizer`.

In [ ]:
class Optimizer:
    """Абстрактный базовый класс для оптимизаторов."""
    def __init__(self, name: str):
        self.name = name

    def optimize(self, func: Callable, bounds: List[Tuple[float, float]], **kwargs) -> Tuple[Any, float]:
        raise NotImplementedError("Метод optimize должен быть реализован в подклассе.")

class MonteCarloOptimizer(Optimizer):
    """Оптимизация методом Монте-Карло"""
    def __init__(self, iterations: int = 1000):
        super().__init__("monte_carlo")
        self.iterations = iterations

    def optimize(self, func: Callable, bounds: List[Tuple[float, float]], **kwargs) -> Tuple[List[float], float]:
        best_point = None
        best_value = float('inf')
        dim = len(bounds)
        for _ in range(self.iterations):
            # Генерируем случайную точку внутри границ
            point = [random.uniform(b[0], b[1]) for b in bounds]
            val = func(point)
            if val < best_value:
                best_value = val
                best_point = point
        return best_point, best_value

class DifferentialEvolutionOptimizer(Optimizer):
    """Оптимизация методом дифференциальной эволюции"""
    def __init__(self, maxiter: int = 100):
        super().__init__("differential_evolution")
        self.maxiter = maxiter

    def optimize(self, func: Callable, bounds: List[Tuple[float, float]], **kwargs) -> Tuple[np.ndarray, float]:
        result = differential_evolution(func, bounds, maxiter=self.maxiter, disp=False)
        return result.x, result.fun


## Менеджер для сбора результатов
Класс `ResultManager` собирает и хранит в памяти результаты каждого запуска оптимизатора, без записи в файл.

In [ ]:
class ResultManager:
    """Хранит результаты оптимизации в списке словарей."""
    def __init__(self):
        self.records = []

    def add(self, method_name: str, best_point: Any, best_metric: float) -> None:
        self.records.append({
            'method': method_name,
            'best_point': best_point,
            'best_mse': best_metric
        })

    def get_all(self) -> List[dict]:
        return self.records


## Построение целевой функции
Функция возвращает вложенную функцию, которая принимает набор параметров и вычисляет MSE между эталоном и текущим изображением (здесь без трансформаций).

In [ ]:
def make_objective(reference: np.ndarray, target: np.ndarray) -> Callable:
    """Создаёт функцию, принимающую параметры и возвращающую MSE."""
    def objective(params: List[float]) -> float:
        # Здесь можно добавить преобразования (например, масштаб, поворот и пр.)
        transformed = target  # Без изменений
        return calculate_mse(reference, transformed)
    return objective


## Основной раздел: запуск оптимизаторов
Загружаем изображения, устанавливаем эталон, задаём границы параметров, инициализируем менеджер, затем запускаем оба метода и выводим результаты прямо в блокноте.

In [ ]:
if __name__ == '__main__':
    # Путь к папке с изображениями
    folder = 'save_folder'
    imgs = load_images_from_folder(folder)
    if not imgs:
        print('Папка пуста или нет изображений')
        raise SystemExit

    # Первое изображение используем как эталон
    reference_img = imgs[0]

    # Пример границ для гипотетических параметров (например, масштаб по X и Y)
    bounds = [(0.5, 2.0),  # масштаб по X
              (0.5, 2.0)]  # масштаб по Y

    # Инициализируем менеджер и оптимизаторы
    manager = ResultManager()
    optimizers = [MonteCarloOptimizer(iterations=500), DifferentialEvolutionOptimizer(maxiter=50)]

    # Запуск методов оптимизации
    for opt in optimizers:
        print(f"\n--- Запуск метода: {opt.name} ---")
        obj_func = make_objective(reference_img, reference_img)
        best_pt, best_val = opt.optimize(obj_func, bounds)
        manager.add(opt.name, best_pt, best_val)
        print(f"Лучшие параметры: {best_pt}")
        print(f"Значение MSE: {best_val}")

    # Вывод всех собранных результатов
    print("\n=== Сводка всех результатов ===")
    for rec in manager.get_all():
        print(f"Метод: {rec['method']}, Лучшая точка: {rec['best_point']}, MSE: {rec['best_mse']}")
